In [8]:
datasets_path = "./datasets/PetImages"

In [9]:
from os.path import basename
from typing import Literal

import cv2
import numpy as np
import pandas as pd


class ImagePreProcessing:
    """
    A class for various image pre-processing tasks.
    """

    @staticmethod
    def load_image(image_path: str) -> np.ndarray:
        """Loads an image from the specified path.

        Args:
            image_path (str): The path to the image file.
        Returns:
            np.ndarray: The loaded image as a NumPy array.
        """
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found at path: {image_path}")
        return image

    @staticmethod
    def get_base_name(image_path: str) -> str:
        """Extracts the base name from the image path.

        Args:
            image_path (str): The path to the image file.
        Returns:
            str: The base name of the image file.
        """
        return basename(image_path)

    @staticmethod
    def get_label_from_image_path(
        image_path: str,
        binarizer: bool = False
    ) -> str:
        """Extracts the label from the image path.

        Args:
            image_path (str): The path to the image file.
            binarizer (bool): Whether to binarizer the label.
        Returns:
            str: The extracted label.
        """
        base_name = basename(image_path)
        label = base_name.split('.')[0][-1]
        if binarizer:
            if int(label) > 2:
                label = '1'
            else:
                label = '0'
        return label

    @staticmethod
    def resize_image(
        image: np.ndarray,
        size: tuple[int, int]
    ) -> np.ndarray:
        """Resizes the image to the specified size.

        Args:
            image (np.ndarray): The input image.
            size (tuple): The desired size as (width, height).
        Returns:
            np.ndarray: The resized image.
        """
        return cv2.resize(image, size)

    @staticmethod
    def normalize_image(image: np.ndarray) -> np.ndarray:
        """Normalizes the image pixel values to the range [0, 1].

        Args:
            image (np.ndarray): The input image.
        Returns:
            np.ndarray: The normalized image.
        """
        return image.astype(np.float32) / 255.0

    @staticmethod
    def to_grayscale(image: np.ndarray) -> np.ndarray:
        """Converts the image to grayscale.

        Args:
            image (np.ndarray): The input image.
        Returns:
            np.ndarray: The grayscale image.
        """
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    @staticmethod
    def save_image(image: np.ndarray, save_path: str) -> None:
        """Saves the image to the specified path.

        Args:
            image (np.ndarray): The image to save.
            save_path (str): The path to save the image.
        """
        cv2.imwrite(save_path, image)

    @staticmethod
    def rotate_image(
        image: np.ndarray,
        angle: float
    ) -> np.ndarray:
        """Rotates the image by the specified angle.

        Args:
            image (np.ndarray): The input image.
            angle (float): The angle to rotate the image.
        Returns:
            np.ndarray: The rotated image.
        """
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h))

    @staticmethod
    def flip_image(
        image: np.ndarray,
        direction: str
    ) -> np.ndarray:
        """Flips the image in the specified direction.

        Args:
            image (np.ndarray): The input image.
            direction (str): The direction to flip
                ('horizontal' or 'vertical').
        Returns:
            np.ndarray: The flipped image.
        """
        if direction == 'horizontal':
            return cv2.flip(image, 1)
        elif direction == 'vertical':
            return cv2.flip(image, 0)
        else:
            raise ValueError("Direction must be 'horizontal' or 'vertical'")

    def crop_image(
        self,
        image: np.ndarray,
        x: int,
        y: int,
        width: int,
        height: int
    ) -> np.ndarray:
        """Crops the image to the specified rectangle.

        Args:
            image (np.ndarray): The input image.
            x (int): The x-coordinate of the top-left corner.
            y (int): The y-coordinate of the top-left corner.
            width (int): The width of the crop rectangle.
            height (int): The height of the crop rectangle.
        Returns:
            np.ndarray: The cropped image.
        """
        return image[y:y + height, x:x + width]

    @staticmethod
    def convert_color_space(
        image: np.ndarray,
        color_space: Literal['HSV', 'LAB', 'YCrCb']
    ) -> np.ndarray:
        """Converts the image to the specified color space.

        Args:
            image (np.ndarray): The input image.
            color_space (str): The target color space
                ('HSV', 'LAB', 'YCrCb', etc.).
        Returns:
            np.ndarray: The image in the new color space.
        """
        if color_space == 'HSV':
            return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        elif color_space == 'LAB':
            return cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        elif color_space == 'YCrCb':
            return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        else:
            raise ValueError("Unsupported color space")

    @staticmethod
    def apply_gaussian_blur(
        image: np.ndarray,
        kernel_size: tuple[int, int]
    ) -> np.ndarray:
        """Applies Gaussian blur to the image.

        Args:
            image (np.ndarray): The input image.
            kernel_size (tuple): The size of the Gaussian kernel.
        Returns:
            np.ndarray: The blurred image.
        """
        return cv2.GaussianBlur(image, kernel_size, 0)

    @staticmethod
    def adjust_brightness_contrast(
        image: np.ndarray,
        brightness: int = 0,
        contrast: int = 0
    ) -> np.ndarray:
        """Adjusts the brightness and contrast of the image.

        Args:
            image (np.ndarray): The input image.
            brightness (int): The brightness adjustment value.
            contrast (int): The contrast adjustment value.
        Returns:
            np.ndarray: The adjusted image.
        """
        buf = cv2.convertScaleAbs(
            image, alpha=1 + contrast / 100.0, beta=brightness
        )
        return buf

    @staticmethod
    def histogram_equalization(image: np.ndarray) -> np.ndarray:
        """Applies histogram equalization to the image.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The image after histogram equalization.
        """
        if len(image.shape) == 2:
            return cv2.equalizeHist(image)
        else:
            raise ValueError(
                "Histogram equalization requires a grayscale image"
            )

    @staticmethod
    def images_to_rgb_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of RGB images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of RGB images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data. Default
                is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_red",
            "mean_green",
            "mean_blue",
            "std_red",
            "std_green",
            "std_blue"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if img.shape[2] != 3:
                raise ValueError("All images must be RGB.")

            if normalize:
                img_f = img.astype(np.float32) / 255.0
            else:
                img_f = img.astype(np.float32)
            mean_red = np.mean(img_f[:, :, 0])
            mean_green = np.mean(img_f[:, :, 1])
            mean_blue = np.mean(img_f[:, :, 2])
            std_red = np.std(img_f[:, :, 0])
            std_green = np.std(img_f[:, :, 1])
            std_blue = np.std(img_f[:, :, 2])
            # flattened_image = img.flatten()
            row = [
                # flattened_image,
                name,
                label,
                mean_red,
                mean_green,
                mean_blue,
                std_red,
                std_green,
                std_blue
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def rgb_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_red"].values,
            df["mean_green"].values,
            df["mean_blue"].values,
            df["std_red"].values,
            df["std_green"].values,
            df["std_blue"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def csv_to_dataframe(
        csv_path: str
    ) -> pd.DataFrame:
        """Loads a CSV file and returns it as a pandas DataFrame.

        Args:
            csv_path (str): The path to the CSV file.

        Returns:
            pd.DataFrame: The loaded DataFrame.
        """
        try:
            return pd.read_csv(csv_path)
        except Exception as e:
            raise RuntimeError(f"Failed to load CSV file: {e}")

    @staticmethod
    def dataframe_to_csv(
        df: pd.DataFrame,
        save_path: str
    ) -> None:
        """Saves the DataFrame to a CSV file.

        Args:
            df (pd.DataFrame): The DataFrame to save.
            save_path (str): The path to save the CSV file.
        """
        df.to_csv(save_path, index=False)

    @staticmethod
    def segment_to_padding_image(
        image: np.ndarray,
        target_size: tuple[int, int]
    ) -> np.ndarray:
        """Segments the image and pads it to the target size.

        Args:
            image (np.ndarray): The input image.
            target_size (tuple): The desired size as (width, height).
        Returns:
            np.ndarray: The segmented and padded image.
        """
        h, w = image.shape[:2]
        target_w, target_h = target_size

        scale = min(target_w / w, target_h / h)
        new_w = int(w * scale)
        new_h = int(h * scale)

        resized_image = cv2.resize(image, (new_w, new_h))

        padded_image = np.zeros((target_h, target_w, 3), dtype=image.dtype)
        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
        padded_image[
            y_offset:y_offset + new_h, x_offset:x_offset + new_w
        ] = resized_image

        return padded_image

    @staticmethod
    def segmented_image_to_padding_image(
        image: np.ndarray,
        target_size: tuple[int, int],
        pad_color: tuple[int, int, int] = (0, 0, 0)
    ) -> np.ndarray:
        """Segments the image and pads it to the target size.

        Args:
            image (np.ndarray): The input image.
            target_size (tuple): The desired size as (width, height).
            pad_color (tuple): RGB color used for padding areas
                (default black).
        Returns:
            np.ndarray: The segmented and padded image.
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        if not contours:
            raise ValueError("No contours found in the image.")
        # use the largest contour to capture the main ROI
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)

        if w == 0 or h == 0:
            raise ValueError("Invalid bounding box with zero width or height.")

        segmented_image = image[y:y + h, x:x + w]

        h_seg, w_seg = segmented_image.shape[:2]
        target_w, target_h = target_size

        if target_w <= 0 or target_h <= 0:
            raise ValueError("target_size must contain positive integers.")

        scale = min(target_w / w_seg, target_h / h_seg)
        new_w = max(1, int(w_seg * scale))
        new_h = max(1, int(h_seg * scale))

        resized_image = cv2.resize(
            segmented_image, (new_w, new_h), interpolation=cv2.INTER_AREA
        )

        # create padded image filled with pad_color
        padded_image = np.zeros((target_h, target_w, 3), dtype=image.dtype)
        padded_image[:, :] = pad_color

        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
        padded_image[
            y_offset:y_offset + new_h, x_offset:x_offset + new_w
        ] = resized_image

        return padded_image

    @staticmethod
    def augment_image_with_noise(
        image: np.ndarray,
        noise_type: Literal['gaussian', 'salt_and_pepper'],
        amount: float = 0.004
    ) -> np.ndarray:
        """Augments the image by adding noise.

        Args:
            image (np.ndarray): The input image.
            noise_type (str): The type of noise to add
                ('gaussian' or 'salt_and_pepper').
            amount (float): The amount of noise to add (default is 0.004).
        Returns:
            np.ndarray: The augmented image with noise.
        """
        if noise_type == 'gaussian':
            row, col, ch = image.shape
            mean = 0
            sigma = 25
            gauss = np.random.normal(mean, sigma, (row, col, ch))
            gauss = gauss.reshape(row, col, ch)
            noisy = image + gauss
            noisy = np.clip(noisy, 0, 255).astype(np.uint8)
            return noisy

        elif noise_type == 'salt_and_pepper':
            noisy = np.copy(image)
            num_salt = np.ceil(amount * image.size * 0.5)
            coords = [
                np.random.randint(0, i - 1, int(num_salt))
                for i in image.shape
            ]
            noisy[coords[0], coords[1], :] = 255

            num_pepper = np.ceil(amount * image.size * 0.5)
            coords = [
                np.random.randint(0, i - 1, int(num_pepper))
                for i in image.shape
            ]
            noisy[coords[0], coords[1], :] = 0
            return noisy

        else:
            raise ValueError("Unsupported noise type")

    def equalize_color_image(
        self,
        image: np.ndarray
    ) -> np.ndarray:
        """Applies histogram equalization to each channel of a color image.

        Args:
            image (np.ndarray): The input color image.
        Returns:
            np.ndarray: The color image after histogram equalization.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be a color image (3 channels).")

        channels = cv2.split(image)
        eq_channels = [
            cv2.equalizeHist(ch) for ch in channels
        ]
        equalized_image = cv2.merge(eq_channels)
        return equalized_image

    @staticmethod
    def rgb_image_to_hsv_image(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an RGB image to HSV color space.

        Args:
            image (np.ndarray): The input RGB image.
        Returns:
            np.ndarray: The image in HSV color space.
        """
        return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    @staticmethod
    def hsv_image_to_rbg_image(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an HSV image to RGB color space.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            np.ndarray: The image in RGB color space.
        """
        return cv2.cvtColor(image, cv2.COLOR_HSV2RGB)

    @staticmethod
    def hsv_images_to_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of HSV images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of HSV images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = ["image", "class"] + [
            f"pixel_{i}" for i in range(images[0].size)
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 3 or img.shape[2] != 3:
                raise ValueError("All images must be HSV.")

            flattened_image = img.flatten()
            if normalize:
                flattened_image = flattened_image.astype(np.float32) / 255.0
            row = [name, label] + flattened_image.tolist()
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def hsv_images_to_summarized_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of HSV images to a summarized pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of HSV images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_H",
            "std_H",
            "mean_S",
            "std_S",
            "mean_V",
            "std_V"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 3 or img.shape[2] != 3:
                raise ValueError("All images must be HSV.")

            h_channel = img[:, :, 0]
            s_channel = img[:, :, 1]
            v_channel = img[:, :, 2]

            if normalize:
                h_channel = h_channel.astype(np.float32) / 255.0
                s_channel = s_channel.astype(np.float32) / 255.0
                v_channel = v_channel.astype(np.float32) / 255.0

            mean_H = np.mean(h_channel)
            std_H = np.std(h_channel)
            mean_S = np.mean(s_channel)
            std_S = np.std(s_channel)
            mean_V = np.mean(v_channel)
            std_V = np.std(v_channel)

            row = [
                name,
                label,
                mean_H,
                std_H,
                mean_S,
                std_S,
                mean_V,
                std_V
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def hsv_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the HSV DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        pixel_columns = [
            col for col in df.columns
            if col.startswith("pixel_")
        ]
        X = df[pixel_columns].values  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def hsv_summarized_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the summarized HSV DataFrame into features (X) and labels
        (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_H"].values,
            df["std_H"].values,
            df["mean_S"].values,
            df["std_S"].values,
            df["mean_V"].values,
            df["std_V"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def grayscale_images_to_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of grayscale images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of grayscale images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = ["image", "class"] + [
            f"pixel_{i}" for i in range(images[0].size)
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 2:
                raise ValueError("All images must be grayscale.")

            flattened_image = img.flatten()
            if normalize:
                flattened_image = flattened_image.astype(np.float32) / 255.0
            row = [name, label] + flattened_image.tolist()
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def grayscale_images_to_summarized_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of grayscale images to a summarized pandas
        DataFrame.

        Args:
            images (list[np.ndarray]): List of grayscale images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: Summarized DataFrame with mean and std
                deviation of pixel values.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_pixel",
            "std_pixel"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 2:
                raise ValueError("All images must be grayscale.")

            if normalize:
                img_f = img.astype(np.float32) / 255.0
            else:
                img_f = img.astype(np.float32)
            mean_pixel = np.mean(img_f)
            std_pixel = np.std(img_f)
            row = [
                name,
                label,
                mean_pixel,
                std_pixel
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def grayscale_to_rgb(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts a grayscale image to RGB by stacking the grayscale
        channel.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The converted RGB image.
        """
        if len(image.shape) != 2:
            raise ValueError("Input image must be grayscale.")
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    @staticmethod
    def grayscale_to_hsv(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts a grayscale image to HSV by stacking the grayscale
        channel.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The converted HSV image.
        """
        if len(image.shape) != 2:
            raise ValueError("Input image must be grayscale.")
        rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    @staticmethod
    def hsv_to_grayscale(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an HSV image to grayscale.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            np.ndarray: The converted grayscale image.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be HSV.")
        rgb_image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

    @staticmethod
    def split_hsv_channels(
        image: np.ndarray
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Splits an HSV image into its H, S, and V channels.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            tuple[np.ndarray, np.ndarray, np.ndarray]: The H, S, and V
                channels.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be HSV.")
        h, s, v = cv2.split(image)
        return h, s, v

    @staticmethod
    def merge_hsv_channels(
        h: np.ndarray,
        s: np.ndarray,
        v: np.ndarray
    ) -> np.ndarray:
        """Merges H, S, and V channels into an HSV image.

        Args:
            h (np.ndarray): The H channel.
            s (np.ndarray): The S channel.
            v (np.ndarray): The V channel.
        Returns:
            np.ndarray: The merged HSV image.
        """
        return cv2.merge([h, s, v])

    @staticmethod
    def grayscale_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the grayscale DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        pixel_columns = [
            col for col in df.columns
            if col.startswith("pixel_")
        ]
        X = df[pixel_columns].values  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def grayscale_summarized_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the summarized grayscale DataFrame into features (X)
        and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_pixel"].values,
            df["std_pixel"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

In [10]:
import cv2
import numpy as np
import pandas as pd


class ImagePreProcessing:
    """
    A class for various image pre-processing tasks.
    """

    @staticmethod
    def load_image(image_path: str) -> np.ndarray:
        """Loads an image from the specified path.

        Args:
            image_path (str): The path to the image file.
        Returns:
            np.ndarray: The loaded image as a NumPy array.
        """
        image = cv2.imread(image_path)
        if image is None:
            raise FileNotFoundError(f"Image not found at path: {image_path}")
        return image

    @staticmethod
    def get_base_name(image_path: str) -> str:
        """Extracts the base name from the image path.

        Args:
            image_path (str): The path to the image file.
        Returns:
            str: The base name of the image file.
        """
        return basename(image_path)

    @staticmethod
    def get_label_from_image_path(
        image_path: str,
        binarizer: bool = False
    ) -> str:
        """Extracts the label from the image path.

        Args:
            image_path (str): The path to the image file.
            binarizer (bool): Whether to binarizer the label.
        Returns:
            str: The extracted label.
        """
        base_name = basename(image_path)
        label = base_name.split('.')[0][-1]
        if binarizer:
            if int(label) > 2:
                label = '1'
            else:
                label = '0'
        return label

    @staticmethod
    def resize_image(
        image: np.ndarray,
        size: tuple[int, int]
    ) -> np.ndarray:
        """Resizes the image to the specified size.

        Args:
            image (np.ndarray): The input image.
            size (tuple): The desired size as (width, height).
        Returns:
            np.ndarray: The resized image.
        """
        return cv2.resize(image, size)

    @staticmethod
    def normalize_image(image: np.ndarray) -> np.ndarray:
        """Normalizes the image pixel values to the range [0, 1].

        Args:
            image (np.ndarray): The input image.
        Returns:
            np.ndarray: The normalized image.
        """
        return image.astype(np.float32) / 255.0

    @staticmethod
    def to_grayscale(image: np.ndarray) -> np.ndarray:
        """Converts the image to grayscale.

        Args:
            image (np.ndarray): The input image.
        Returns:
            np.ndarray: The grayscale image.
        """
        return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    @staticmethod
    def save_image(image: np.ndarray, save_path: str) -> None:
        """Saves the image to the specified path.

        Args:
            image (np.ndarray): The image to save.
            save_path (str): The path to save the image.
        """
        cv2.imwrite(save_path, image)

    @staticmethod
    def rotate_image(
        image: np.ndarray,
        angle: float
    ) -> np.ndarray:
        """Rotates the image by the specified angle.

        Args:
            image (np.ndarray): The input image.
            angle (float): The angle to rotate the image.
        Returns:
            np.ndarray: The rotated image.
        """
        (h, w) = image.shape[:2]
        center = (w // 2, h // 2)
        M = cv2.getRotationMatrix2D(center, angle, 1.0)
        return cv2.warpAffine(image, M, (w, h))

    @staticmethod
    def flip_image(
        image: np.ndarray,
        direction: str
    ) -> np.ndarray:
        """Flips the image in the specified direction.

        Args:
            image (np.ndarray): The input image.
            direction (str): The direction to flip
                ('horizontal' or 'vertical').
        Returns:
            np.ndarray: The flipped image.
        """
        if direction == 'horizontal':
            return cv2.flip(image, 1)
        elif direction == 'vertical':
            return cv2.flip(image, 0)
        else:
            raise ValueError("Direction must be 'horizontal' or 'vertical'")

    def crop_image(
        self,
        image: np.ndarray,
        x: int,
        y: int,
        width: int,
        height: int
    ) -> np.ndarray:
        """Crops the image to the specified rectangle.

        Args:
            image (np.ndarray): The input image.
            x (int): The x-coordinate of the top-left corner.
            y (int): The y-coordinate of the top-left corner.
            width (int): The width of the crop rectangle.
            height (int): The height of the crop rectangle.
        Returns:
            np.ndarray: The cropped image.
        """
        return image[y:y + height, x:x + width]

    @staticmethod
    def convert_color_space(
        image: np.ndarray,
        color_space: Literal['HSV', 'LAB', 'YCrCb']
    ) -> np.ndarray:
        """Converts the image to the specified color space.

        Args:
            image (np.ndarray): The input image.
            color_space (str): The target color space
                ('HSV', 'LAB', 'YCrCb', etc.).
        Returns:
            np.ndarray: The image in the new color space.
        """
        if color_space == 'HSV':
            return cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
        elif color_space == 'LAB':
            return cv2.cvtColor(image, cv2.COLOR_BGR2Lab)
        elif color_space == 'YCrCb':
            return cv2.cvtColor(image, cv2.COLOR_BGR2YCrCb)
        else:
            raise ValueError("Unsupported color space")

    @staticmethod
    def apply_gaussian_blur(
        image: np.ndarray,
        kernel_size: tuple[int, int]
    ) -> np.ndarray:
        """Applies Gaussian blur to the image.

        Args:
            image (np.ndarray): The input image.
            kernel_size (tuple): The size of the Gaussian kernel.
        Returns:
            np.ndarray: The blurred image.
        """
        return cv2.GaussianBlur(image, kernel_size, 0)

    @staticmethod
    def adjust_brightness_contrast(
        image: np.ndarray,
        brightness: int = 0,
        contrast: int = 0
    ) -> np.ndarray:
        """Adjusts the brightness and contrast of the image.

        Args:
            image (np.ndarray): The input image.
            brightness (int): The brightness adjustment value.
            contrast (int): The contrast adjustment value.
        Returns:
            np.ndarray: The adjusted image.
        """
        buf = cv2.convertScaleAbs(
            image, alpha=1 + contrast / 100.0, beta=brightness
        )
        return buf

    @staticmethod
    def histogram_equalization(image: np.ndarray) -> np.ndarray:
        """Applies histogram equalization to the image.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The image after histogram equalization.
        """
        if len(image.shape) == 2:
            return cv2.equalizeHist(image)
        else:
            raise ValueError(
                "Histogram equalization requires a grayscale image"
            )

    @staticmethod
    def images_to_rgb_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of RGB images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of RGB images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data. Default
                is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_red",
            "mean_green",
            "mean_blue",
            "std_red",
            "std_green",
            "std_blue"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if img.shape[2] != 3:
                raise ValueError("All images must be RGB.")

            if normalize:
                img_f = img.astype(np.float32) / 255.0
            else:
                img_f = img.astype(np.float32)
            mean_red = np.mean(img_f[:, :, 0])
            mean_green = np.mean(img_f[:, :, 1])
            mean_blue = np.mean(img_f[:, :, 2])
            std_red = np.std(img_f[:, :, 0])
            std_green = np.std(img_f[:, :, 1])
            std_blue = np.std(img_f[:, :, 2])
            # flattened_image = img.flatten()
            row = [
                # flattened_image,
                name,
                label,
                mean_red,
                mean_green,
                mean_blue,
                std_red,
                std_green,
                std_blue
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def rgb_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_red"].values,
            df["mean_green"].values,
            df["mean_blue"].values,
            df["std_red"].values,
            df["std_green"].values,
            df["std_blue"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def csv_to_dataframe(
        csv_path: str
    ) -> pd.DataFrame:
        """Loads a CSV file and returns it as a pandas DataFrame.

        Args:
            csv_path (str): The path to the CSV file.

        Returns:
            pd.DataFrame: The loaded DataFrame.
        """
        try:
            return pd.read_csv(csv_path)
        except Exception as e:
            raise RuntimeError(f"Failed to load CSV file: {e}")

    @staticmethod
    def dataframe_to_csv(
        df: pd.DataFrame,
        save_path: str
    ) -> None:
        """Saves the DataFrame to a CSV file.

        Args:
            df (pd.DataFrame): The DataFrame to save.
            save_path (str): The path to save the CSV file.
        """
        df.to_csv(save_path, index=False)

    @staticmethod
    def segment_to_padding_image(
        image: np.ndarray,
        target_size: tuple[int, int]
    ) -> np.ndarray:
        """Segments the image and pads it to the target size.

        Args:
            image (np.ndarray): The input image.
            target_size (tuple): The desired size as (width, height).
        Returns:
            np.ndarray: The segmented and padded image.
        """
        h, w = image.shape[:2]
        target_w, target_h = target_size

        scale = min(target_w / w, target_h / h)
        new_w = int(w * scale)
        new_h = int(h * scale)

        resized_image = cv2.resize(image, (new_w, new_h))

        padded_image = np.zeros((target_h, target_w, 3), dtype=image.dtype)
        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
        padded_image[
            y_offset:y_offset + new_h, x_offset:x_offset + new_w
        ] = resized_image

        return padded_image

    @staticmethod
    def segmented_image_to_padding_image(
        image: np.ndarray,
        target_size: tuple[int, int],
        pad_color: tuple[int, int, int] = (0, 0, 0)
    ) -> np.ndarray:
        """Segments the image and pads it to the target size.

        Args:
            image (np.ndarray): The input image.
            target_size (tuple): The desired size as (width, height).
            pad_color (tuple): RGB color used for padding areas
                (default black).
        Returns:
            np.ndarray: The segmented and padded image.
        """
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 1, 255, cv2.THRESH_BINARY)
        contours, _ = cv2.findContours(
            thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
        )

        if not contours:
            raise ValueError("No contours found in the image.")
        # use the largest contour to capture the main ROI
        c = max(contours, key=cv2.contourArea)
        x, y, w, h = cv2.boundingRect(c)

        if w == 0 or h == 0:
            raise ValueError("Invalid bounding box with zero width or height.")

        segmented_image = image[y:y + h, x:x + w]

        h_seg, w_seg = segmented_image.shape[:2]
        target_w, target_h = target_size

        if target_w <= 0 or target_h <= 0:
            raise ValueError("target_size must contain positive integers.")

        scale = min(target_w / w_seg, target_h / h_seg)
        new_w = max(1, int(w_seg * scale))
        new_h = max(1, int(h_seg * scale))

        resized_image = cv2.resize(
            segmented_image, (new_w, new_h), interpolation=cv2.INTER_AREA
        )

        # create padded image filled with pad_color
        padded_image = np.zeros((target_h, target_w, 3), dtype=image.dtype)
        padded_image[:, :] = pad_color

        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
        padded_image[
            y_offset:y_offset + new_h, x_offset:x_offset + new_w
        ] = resized_image

        return padded_image

    @staticmethod
    def augment_image_with_noise(
        image: np.ndarray,
        noise_type: Literal['gaussian', 'salt_and_pepper'],
        amount: float = 0.004
    ) -> np.ndarray:
        """Augments the image by adding noise.

        Args:
            image (np.ndarray): The input image.
            noise_type (str): The type of noise to add
                ('gaussian' or 'salt_and_pepper').
            amount (float): The amount of noise to add (default is 0.004).
        Returns:
            np.ndarray: The augmented image with noise.
        """
        if noise_type == 'gaussian':
            row, col, ch = image.shape
            mean = 0
            sigma = 25
            gauss = np.random.normal(mean, sigma, (row, col, ch))
            gauss = gauss.reshape(row, col, ch)
            noisy = image + gauss
            noisy = np.clip(noisy, 0, 255).astype(np.uint8)
            return noisy

        elif noise_type == 'salt_and_pepper':
            noisy = np.copy(image)
            num_salt = np.ceil(amount * image.size * 0.5)
            coords = [
                np.random.randint(0, i - 1, int(num_salt))
                for i in image.shape
            ]
            noisy[coords[0], coords[1], :] = 255

            num_pepper = np.ceil(amount * image.size * 0.5)
            coords = [
                np.random.randint(0, i - 1, int(num_pepper))
                for i in image.shape
            ]
            noisy[coords[0], coords[1], :] = 0
            return noisy

        else:
            raise ValueError("Unsupported noise type")

    def equalize_color_image(
        self,
        image: np.ndarray
    ) -> np.ndarray:
        """Applies histogram equalization to each channel of a color image.

        Args:
            image (np.ndarray): The input color image.
        Returns:
            np.ndarray: The color image after histogram equalization.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be a color image (3 channels).")

        channels = cv2.split(image)
        eq_channels = [
            cv2.equalizeHist(ch) for ch in channels
        ]
        equalized_image = cv2.merge(eq_channels)
        return equalized_image

    @staticmethod
    def rgb_image_to_hsv_image(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an RGB image to HSV color space.

        Args:
            image (np.ndarray): The input RGB image.
        Returns:
            np.ndarray: The image in HSV color space.
        """
        return cv2.cvtColor(image, cv2.COLOR_RGB2HSV)

    @staticmethod
    def hsv_image_to_rbg_image(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an HSV image to RGB color space.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            np.ndarray: The image in RGB color space.
        """
        return cv2.cvtColor(image, cv2.COLOR_HSV2RGB)

    @staticmethod
    def hsv_images_to_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of HSV images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of HSV images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = ["image", "class"] + [
            f"pixel_{i}" for i in range(images[0].size)
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 3 or img.shape[2] != 3:
                raise ValueError("All images must be HSV.")

            flattened_image = img.flatten()
            if normalize:
                flattened_image = flattened_image.astype(np.float32) / 255.0
            row = [name, label] + flattened_image.tolist()
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def hsv_images_to_summarized_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of HSV images to a summarized pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of HSV images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_H",
            "std_H",
            "mean_S",
            "std_S",
            "mean_V",
            "std_V"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 3 or img.shape[2] != 3:
                raise ValueError("All images must be HSV.")

            h_channel = img[:, :, 0]
            s_channel = img[:, :, 1]
            v_channel = img[:, :, 2]

            if normalize:
                h_channel = h_channel.astype(np.float32) / 255.0
                s_channel = s_channel.astype(np.float32) / 255.0
                v_channel = v_channel.astype(np.float32) / 255.0

            mean_H = np.mean(h_channel)
            std_H = np.std(h_channel)
            mean_S = np.mean(s_channel)
            std_S = np.std(s_channel)
            mean_V = np.mean(v_channel)
            std_V = np.std(v_channel)

            row = [
                name,
                label,
                mean_H,
                std_H,
                mean_S,
                std_S,
                mean_V,
                std_V
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def hsv_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the HSV DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        pixel_columns = [
            col for col in df.columns
            if col.startswith("pixel_")
        ]
        X = df[pixel_columns].values  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def hsv_summarized_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the summarized HSV DataFrame into features (X) and labels
        (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_H"].values,
            df["std_H"].values,
            df["mean_S"].values,
            df["std_S"].values,
            df["mean_V"].values,
            df["std_V"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def grayscale_images_to_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of grayscale images to a pandas DataFrame.

        Args:
            images (list[np.ndarray]): List of grayscale images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: DataFrame where each row represents an image
                flattened into a 1D array.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = ["image", "class"] + [
            f"pixel_{i}" for i in range(images[0].size)
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 2:
                raise ValueError("All images must be grayscale.")

            flattened_image = img.flatten()
            if normalize:
                flattened_image = flattened_image.astype(np.float32) / 255.0
            row = [name, label] + flattened_image.tolist()
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def grayscale_images_to_summarized_dataframe(
        images: list[np.ndarray],
        labels: list[str],
        names: list[str],
        normalize: bool = False
    ) -> pd.DataFrame:
        """Converts a list of grayscale images to a summarized pandas
        DataFrame.

        Args:
            images (list[np.ndarray]): List of grayscale images.
            labels (list[str]): Corresponding labels for the images.
            names (list[str]): Corresponding names for the images.
            normalize (bool): Whether to normalize image data.
                Default is False.
        Returns:
            pd.DataFrame: Summarized DataFrame with mean and std
                deviation of pixel values.
        """
        if len(images) != len(labels) or len(images) != len(names):
            raise ValueError(
                "The number of images must match the number of labels"
                " and names."
            )
        columns = [
            "image",
            "class",
            "mean_pixel",
            "std_pixel"
        ]

        data = []
        for img, label, name in zip(images, labels, names):
            if len(img.shape) != 2:
                raise ValueError("All images must be grayscale.")

            if normalize:
                img_f = img.astype(np.float32) / 255.0
            else:
                img_f = img.astype(np.float32)
            mean_pixel = np.mean(img_f)
            std_pixel = np.std(img_f)
            row = [
                name,
                label,
                mean_pixel,
                std_pixel
            ]
            data.append(row)

        df = pd.DataFrame(data, columns=columns)
        return df

    @staticmethod
    def grayscale_to_rgb(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts a grayscale image to RGB by stacking the grayscale
        channel.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The converted RGB image.
        """
        if len(image.shape) != 2:
            raise ValueError("Input image must be grayscale.")
        return cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

    @staticmethod
    def grayscale_to_hsv(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts a grayscale image to HSV by stacking the grayscale
        channel.

        Args:
            image (np.ndarray): The input grayscale image.
        Returns:
            np.ndarray: The converted HSV image.
        """
        if len(image.shape) != 2:
            raise ValueError("Input image must be grayscale.")
        rgb_image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
        return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2HSV)

    @staticmethod
    def hsv_to_grayscale(
        image: np.ndarray
    ) -> np.ndarray:
        """Converts an HSV image to grayscale.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            np.ndarray: The converted grayscale image.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be HSV.")
        rgb_image = cv2.cvtColor(image, cv2.COLOR_HSV2RGB)
        return cv2.cvtColor(rgb_image, cv2.COLOR_RGB2GRAY)

    @staticmethod
    def split_hsv_channels(
        image: np.ndarray
    ) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        """Splits an HSV image into its H, S, and V channels.

        Args:
            image (np.ndarray): The input HSV image.
        Returns:
            tuple[np.ndarray, np.ndarray, np.ndarray]: The H, S, and V
                channels.
        """
        if len(image.shape) != 3 or image.shape[2] != 3:
            raise ValueError("Input image must be HSV.")
        h, s, v = cv2.split(image)
        return h, s, v

    @staticmethod
    def merge_hsv_channels(
        h: np.ndarray,
        s: np.ndarray,
        v: np.ndarray
    ) -> np.ndarray:
        """Merges H, S, and V channels into an HSV image.

        Args:
            h (np.ndarray): The H channel.
            s (np.ndarray): The S channel.
            v (np.ndarray): The V channel.
        Returns:
            np.ndarray: The merged HSV image.
        """
        return cv2.merge([h, s, v])

    @staticmethod
    def grayscale_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the grayscale DataFrame into features (X) and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        pixel_columns = [
            col for col in df.columns
            if col.startswith("pixel_")
        ]
        X = df[pixel_columns].values  # type: ignore
        y = df["class"].values
        return X, y

    @staticmethod
    def grayscale_summarized_dataframe_to_xy(  # type: ignore
        df: pd.DataFrame
    ):
        """Splits the summarized grayscale DataFrame into features (X)
        and labels (y).

        Args:
            df (pd.DataFrame): The input DataFrame.
        Returns:
            tuple[np.ndarray, np.ndarray]: A tuple containing the features
                array (X) and labels array (y).
        """
        characters = [
            df["mean_pixel"].values,
            df["std_pixel"].values
        ]
        X = np.column_stack(characters)  # type: ignore
        y = df["class"].values
        return X, y

In [11]:

from glob import glob
images_dogs_path = glob(datasets_path+"/dogs/*.jpg")
images_cats_path = glob(datasets_path+"/cats/*.jpg")

In [12]:

images_dogs = [
    ImagePreProcessing.load_image(path)
    for path in images_dogs_path[0:100]
]
images_cats = [
    ImagePreProcessing.load_image(path)
    for path in images_cats_path[0:100]
]

In [13]:
images_cats

[array([[[ 20,  33,  19],
         [ 25,  35,  23],
         [ 25,  31,  26],
         ...,
         [ 56,  68,  74],
         [ 31,  49,  50],
         [ 30,  51,  49]],
 
        [[ 20,  33,  19],
         [ 25,  34,  24],
         [ 26,  32,  27],
         ...,
         [ 44,  56,  58],
         [ 28,  42,  40],
         [ 34,  51,  48]],
 
        [[ 22,  33,  17],
         [ 23,  33,  21],
         [ 26,  32,  27],
         ...,
         [ 43,  59,  58],
         [ 31,  50,  47],
         [ 32,  51,  48]],
 
        ...,
 
        [[ 52,  74,  79],
         [ 53,  73,  78],
         [ 53,  72,  77],
         ...,
         [104, 123, 126],
         [100, 124, 122],
         [102, 129, 126]],
 
        [[ 52,  71,  78],
         [ 56,  77,  79],
         [ 56,  77,  74],
         ...,
         [106, 125, 130],
         [101, 124, 126],
         [107, 130, 132]],
 
        [[ 51,  71,  76],
         [ 58,  79,  81],
         [ 57,  77,  78],
         ...,
         [114, 129, 131],
  

In [14]:
images_dogs_gray = [
    ImagePreProcessing.to_grayscale(image)
    for image in images_dogs
]
images_cats_gray = [
    ImagePreProcessing.to_grayscale(image)
    for image in images_cats
]

In [15]:
normalize_dogs=[
    ImagePreProcessing.normalize_image(image)
    for image in images_dogs_gray
]
normalize_cats=[
    ImagePreProcessing.normalize_image(image)
    for image in images_cats_gray
]

In [16]:
summary_dogs= ImagePreProcessing.grayscale_images_to_summarized_dataframe(
    normalize_dogs,
    [0 for _ in range(100)],
    [ImagePreProcessing.get_base_name(image) for image in images_dogs_path[0:100] ])
summary_cats= ImagePreProcessing.grayscale_images_to_summarized_dataframe(
    normalize_cats,
    [1 for _ in range(100)],
    [ImagePreProcessing.get_base_name(image) for image in images_cats_path[0:100] ])


In [17]:
summary_cats

,image,class,mean_pixel,std_pixel
0,7431.jpg,1,0.442980,0.229275
1,2923.jpg,1,0.698873,0.169910
2,10700.jpg,1,0.264578,0.198348
3,7882.jpg,1,0.441039,0.270368
4,6558.jpg,1,0.593944,0.196245
...,...,...,...,...
95,4405.jpg,1,0.441945,0.298402
96,2792.jpg,1,0.510061,0.180253
97,294.jpg,1,0.335087,0.240747
98,10816.jpg,1,0.548925,0.254482


In [18]:
dogsX,dogsY = ImagePreProcessing.grayscale_summarized_dataframe_to_xy(summary_dogs)
catsX,catsY = ImagePreProcessing.grayscale_summarized_dataframe_to_xy(summary_cats)

In [19]:
dogsX

array([[0.4432886 , 0.20944403],
       [0.6021758 , 0.28632447],
       [0.7319963 , 0.2160658 ],
       [0.46631965, 0.20342359],
       [0.4793272 , 0.25535357],
       [0.26471233, 0.15364082],
       [0.33367532, 0.24265409],
       [0.40199673, 0.25499249],
       [0.32000864, 0.19335738],
       [0.6203418 , 0.27880386],
       [0.61837107, 0.17174414],
       [0.25448376, 0.25236037],
       [0.4783744 , 0.21063618],
       [0.37754968, 0.26692754],
       [0.403709  , 0.16829167],
       [0.45994803, 0.1778571 ],
       [0.449087  , 0.24720564],
       [0.39608318, 0.21412338],
       [0.5335537 , 0.18099956],
       [0.5558616 , 0.30564594],
       [0.49859366, 0.1276904 ],
       [0.43919322, 0.18117636],
       [0.5396769 , 0.18695875],
       [0.36978543, 0.19261138],
       [0.49240863, 0.20602252],
       [0.68059295, 0.21297537],
       [0.6685635 , 0.20140576],
       [0.43134758, 0.20603336],
       [0.43267888, 0.2142933 ],
       [0.58257455, 0.23838623],
       [0.

In [20]:
x_dog_train=dogsX[0:80]
x_dog_test=dogsX[80:100]
y_dog_train=dogsY[0:80]
y_dog_test=dogsY[80:100]
x_cat_train=catsX[0:80]
x_cat_test=catsX[80:100]
y_cat_train=catsY[0:80]
y_cat_test=catsY[80:100]

In [21]:
import numpy as np

# Concatenando os dados de Treino
X_train = np.concatenate((x_dog_train, x_cat_train), axis=0)
Y_train = np.concatenate((y_dog_train, y_cat_train), axis=0)

# Concatenando os dados de Teste
X_test = np.concatenate((x_dog_test, x_cat_test), axis=0)
Y_test = np.concatenate((y_dog_test, y_cat_test), axis=0)

In [22]:
print(len(X_train))
print(len(Y_train))
print(len(X_test))
print(len(Y_test))
print(Y_test)

160
160
40
40
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1]


In [ ]:
from sklearn.neural_network import MLPClassifier
clf = MLPClassifier(solver='lbfgs', alpha=1e-5,
                    hidden_layer_sizes=(15,), random_state=1)
clf.fit(X_train, Y_train)

,"hidden_layer_sizes hidden_layer_sizes: array-like of shape(n_layers - 2,), default=(100,)The ith element represents the number of neurons in the ithhidden layer.","(15,)"
,"activation activation: {'identity', 'logistic', 'tanh', 'relu'}, default='relu'Activation function for the hidden layer.- 'identity', no-op activation, useful to implement linear bottleneck, returns f(x) = x- 'logistic', the logistic sigmoid function, returns f(x) = 1 / (1 + exp(-x)).- 'tanh', the hyperbolic tan function, returns f(x) = tanh(x).- 'relu', the rectified linear unit function, returns f(x) = max(0, x)",'relu'
,"solver solver: {'lbfgs', 'sgd', 'adam'}, default='adam'The solver for weight optimization.- 'lbfgs' is an optimizer in the family of quasi-Newton methods.- 'sgd' refers to stochastic gradient descent.- 'adam' refers to a stochastic gradient-based optimizer proposed by Kingma, Diederik, and Jimmy BaFor a comparison between Adam optimizer and SGD, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_training_curves.py`.Note: The default solver 'adam' works pretty well on relativelylarge datasets (with thousands of training samples or more) in terms ofboth training time and validation score.For small datasets, however, 'lbfgs' can converge faster and performbetter.",'lbfgs'
,"alpha alpha: float, default=0.0001Strength of the L2 regularization term. The L2 regularization termis divided by the sample size when added to the loss.For an example usage and visualization of varying regularization, see:ref:`sphx_glr_auto_examples_neural_networks_plot_mlp_alpha.py`.",1e-05
,"batch_size batch_size: int, default='auto'Size of minibatches for stochastic optimizers.If the solver is 'lbfgs', the classifier will not use minibatch.When set to ""auto"", `batch_size=min(200, n_samples)`.",'auto'
,"learning_rate learning_rate: {'constant', 'invscaling', 'adaptive'}, default='constant'Learning rate schedule for weight updates.- 'constant' is a constant learning rate given by 'learning_rate_init'.- 'invscaling' gradually decreases the learning rate at each time step 't' using an inverse scaling exponent of 'power_t'. effective_learning_rate = learning_rate_init / pow(t, power_t)- 'adaptive' keeps the learning rate constant to 'learning_rate_init' as long as training loss keeps decreasing. Each time two consecutive epochs fail to decrease training loss by at least tol, or fail to increase validation score by at least tol if 'early_stopping' is on, the current learning rate is divided by 5.Only used when ``solver='sgd'``.",'constant'
,"learning_rate_init learning_rate_init: float, default=0.001The initial learning rate used. It controls the step-sizein updating the weights. Only used when solver='sgd' or 'adam'.",0.001
,"power_t power_t: float, default=0.5The exponent for inverse scaling learning rate.It is used in updating effective learning rate when the learning_rateis set to 'invscaling'. Only used when solver='sgd'.",0.5
,"max_iter max_iter: int, default=200Maximum number of iterations. The solver iterates until convergence(determined by 'tol') or this number of iterations. For stochasticsolvers ('sgd', 'adam'), note that this determines the number of epochs(how many times each data point will be used), not the number ofgradient steps.",200
,"shuffle shuffle: bool, default=TrueWhether to shuffle samples in each iteration. Only used whensolver='sgd' or 'adam'.",True
,"random_state random_state: int, RandomState instance, default=NoneDetermines random number generation for weights and biasinitialization, train-test split if early stopping is used, and batchsampling when solver='sgd' or 'adam'.Pass an int for reproducible results across multiple function calls.See :term:`Glossary `.",1


In [28]:
Y_pred = clf.predict(X_test)

In [29]:
Y_pred

array([0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0])

In [26]:
Y_test

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [30]:
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    f1_score,
    recall_score,
    cohen_kappa_score
)
accuracy = float(accuracy_score(Y_test, Y_pred))
precision = float(precision_score(Y_test, Y_pred, average='weighted'))
recall = float(recall_score(Y_test, Y_pred, average='weighted'))
f1 = float(f1_score(Y_test, Y_pred, average='weighted'))
kappa = float(cohen_kappa_score(Y_test, Y_pred))

In [32]:
print(accuracy)
print(precision)
print(recall)
print(f1)
print(kappa)

0.375
0.36666666666666664
0.375
0.36507936507936506
-0.25
